In [1]:
import tensorflow as tf
import environment as Env
import keras
import numpy as np
import random

In [2]:
# 딥살사 인공신경망
class DeepSARSA(tf.keras.Model):
    def __init__(self, action_size):
        super(DeepSARSA, self).__init__()
        self.fc1 = keras.layers.Dense(30, activation = 'relu') # 입력층 (unit 개수 : 30, 활성함수 : ReLU)
        self.fc2 = keras.layers.Dense(30, activation = 'relu') # 은닉층 (unit 개수 : 30, 활성함수 : ReLU)
        self.fc_out = keras.layers.Dense(action_size) # 출력층 (action_size = 5, 상, 하, 좌, 우, 제자리)
        
    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.fc_out(x)
        
        return q

In [3]:
# 그리드월드 예제에서의 딥살사 에이전트
class DeepSARSAAgent:
    def __init__(self, state_size, action_size):
        # 상태의 크기와 행동의 크기 정의
        self.state_size = state_size # 상태의 크기 정의
        self.action_size = action_size # 행동의 크기 정의
        
        # 딥살사 하이퍼 파라메터
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.
        self.epsilon_decay = .9999
        self.epsilon_min = 0.01
        self.model = DeepSARSA(self.action_size)
        self.optimizer = keras.optimizers.Adam(lr = self.learning_rate)
        
    # 엡실론 탐욕 정택으로 행동 선택
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            #print('get_action = epsilon :', self.epsilon)
            return random.randrange(self.action_size)
        
        else:
            q_values = self.model(state)
            #print("get_action - q_values : ", q_values)
            return np.argmax(q_values[0])
        
    def train_model(self, state, action, reward, next_state, next_action, done): # epsilon 값이 지정한 최소값보다 큰 경우에 감소시킴
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
        # 학습 파라메터
        model_params = self.model.trainable_variables
        
        with  tf.GradientTape() as tape:
            tape.watch(model_params)
            predict = self.model(state)[0] # 상태를 모델의 입력으로 하여 각 행동에 대한 큐함수 값을 받음
            #print("predict : ", predict)
            one_hot_action = tf.one_hot([action], self.action_size) # 행동을 one hot 인코딩으로 변환
            #print("one_hot_action : ", one_hot_action)
            predict = tf.reduce_sum(one_hot_action * predict, axis = 1) # 현재 행동의 큐함수 값 선택
            #print("predict : ", predict)
            
            # done = True일 경우 에피소드가 끝나서 다음 상태가 없음
            next_q = self.model(next_state)[0][next_action] # 다음 상태를 입력해 다음 행동에 대한 큐함수를 받음
            #print("0. next_q : ", next_q)
            
            target = reward + (1 - done) * self.discount_factor * next_q # DeepSARSA 공식 적용
            #print("10. reward : ", reward)
            #print("11. discount factor : ", self.discount_factor)
            #print("12. target : ", target)
            
            # MSE 오류 함수 계산
            loss = tf.reduce_mean(tf.square(target - predict))
            
        # 오류 함수를 줄이는 방향으로 모델 업데이트
        grads = tape.gradient(loss, model_params) # gradient 계산
        self.optimizer.apply_gradients(zip(grads, model_params)) # gradient를 줄이는 방향으로 가중치 업데이트

In [ ]:
if __name__ == "__main__":
    # 환경과 에이전트 생성
    env = Env.Env(render_speed = 0.01) # 환경 instance 생성 (게임 속도를 0.01로 조정)
    state_size = 15 # 상태 개수 정의
    action_space = [0, 1, 2, 3, 4] # 행동 정의
    action_size = len(action_space) # 행동 개수 정의
    agent = DeepSARSAAgent(state_size, action_size) # DeepSARSAAgnet instance 생성
    
    scores, episodes = [], []
    
    EPISODES = 100 # episode 횟수 정의.
    
    for e in range(EPISODES): 
        print("Episode : ", e + 1)
        done = False
        score = 0
        step = 0
        
        # env 초기화
        #print("-- entering env.reset()")
        state = env.reset() # 환경을 초기화하고 상태를 받음 (list 형식)
        #print("-- exit env.reset()")
        state = np.reshape(state, [1, state_size]) # 상태 list를 (1, 15)의 numpy.array로 변환
        
        while not done: # episode가 끝나지 않으면 계속 실행
            # 몇 번째 스텝인지 확인
            step += 1
            
            # 현재 상태에 대한 행동 선택
            #print("-- entering get_action()")
            action = agent.get_action(state)
            #print("action : ", action)
            #print("-- exit get_action()")
            
            # 선택한 행동으로 환경에서 한 타임스텝 진행 후 샘플 수집
            #print("-- entering env.step()")
            #print("state : ", state)
            next_state, reward, done = env.step(action)
            #print("-- exit env.step()")
            #print("10. next_state : ", next_state)
            #print("11. reward : ", reward)
            next_state = np.reshape(next_state, [1, state_size])            
            #print("11. next_state : ", next_state)
            next_action = agent.get_action(next_state)
            
            # 샘플로 모델 학습
            agent.train_model(state, action, reward, next_state, next_action, done)
            score += reward
            state = next_state
            
        print("step : ", step)
        print("score : ", score)
        print("\n~~~~~~~~~~\n")
        
quit()

Episode :  1
step :  37
score :  -3

~~~~~~~~~~

Episode :  2
step :  43
score :  -2

~~~~~~~~~~

Episode :  3
step :  325
score :  -46

~~~~~~~~~~

Episode :  4
step :  109
score :  -16

~~~~~~~~~~

Episode :  5
step :  175
score :  -21

~~~~~~~~~~

Episode :  6
step :  230
score :  -21

~~~~~~~~~~

Episode :  7
step :  102
score :  -14

~~~~~~~~~~

Episode :  8
step :  205
score :  -24

~~~~~~~~~~

Episode :  9
step :  143
score :  -5

~~~~~~~~~~

Episode :  10
step :  46
score :  -4

~~~~~~~~~~

Episode :  11
step :  263
score :  -17

~~~~~~~~~~

Episode :  12
step :  48
score :  -4

~~~~~~~~~~

Episode :  13
step :  48
score :  -6

~~~~~~~~~~

Episode :  14
step :  78
score :  -6

~~~~~~~~~~

Episode :  15
step :  96
score :  -6

~~~~~~~~~~

Episode :  16
step :  145
score :  -11

~~~~~~~~~~

Episode :  17
step :  65
score :  -5

~~~~~~~~~~

Episode :  18
step :  23
score :  -2

~~~~~~~~~~

Episode :  19
step :  88
score :  -3

~~~~~~~~~~

Episode :  20
step :  51
score :  -3

~~~~

In [ ]:
print("end!")